In [ ]:
# Imports
from datasets import load_dataset
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import random
import pandas as pd
import os, json
from transformers import pipeline
import torch
from torchmetrics.text.rouge import ROUGEScore
from torch.utils.data import Subset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset

In [ ]:
dataset = load_dataset("generative-newsai/news-unmasked")

In [ ]:
train_set = dataset["train"]
test_set = dataset["test"]

## Load the processed train data

In [ ]:
%%script false --no-raise-error

!gdown https://drive.google.com/uc?id=1OUj56f7gX7_C-xQE6YkVBpJICVNcDsNl
!UNZIP_DISABLE_ZIPBOMB_DETECTION=TRUE unzip processed_data.zip

In [ ]:
# Download dataset from https://drive.google.com/file/d/1sE2kae6-Uj7B0ThIX8bTfKdCqq9aKp4n/view?usp=sharing

from datasets import load_from_disk
train_set = load_from_disk("./processed_data")
train_train_set, train_val_set = torch.utils.data.random_split(train_set, 
                                                               [int(0.8 * len(train_set)),
                                                                len(train_set) - int(0.8 * len(train_set))])

# Visualizing the data

In [ ]:
train_set.__len__(), test_set.__len__()

In [ ]:
train_set.features, test_set.features

In [ ]:
train_set[0]

In [ ]:
test_set[random.choice(range(test_set.__len__()))]

In [ ]:
# how many test headlines have [MASK] in them?
mask_token = "[MASK]"
test_with_mask = [x for x in test_set["headline"] if mask_token in x]
print(f"{len(test_with_mask)} out of {len(test_set)} test headlines have [MASK] in them")

In [ ]:
# get tests without [MASK] in them
test_without_mask = [x for x in test_set["headline"] if mask_token not in x]
print(f"{len(test_without_mask)} out of {len(test_set)} test headlines do not have [MASK] in them")

Headlines with no mask in test set

In [ ]:
test_without_mask[random.choice(range(len(test_without_mask)))]

In [ ]:
sections = np.unique(test_set["section"])
print("Unique sections in the test dataset: ", len(sections))

Observations: We have image, section and a masked headline. We need to predict the appropriate word for the masked headline.

In [ ]:
# how many train headlines have [MASK] in them?
train_with_mask = [x for x in train_set["headline"] if mask_token in x]
print(f"{len(train_with_mask)} out of {len(train_set)} train headlines have [MASK] in them")

In [ ]:
sections = np.unique(train_set["section"])
print("Unique sections in the train dataset: ", len(sections))

Observations: Not all the test entries have [MASK] token in them and there are 24 unique sections in the dataset.

## Plot section count

In [ ]:
%%script false --no-raise-error

# Plot np unique with counts
z_train = np.unique(train_set["section"], return_counts=True)
z_test = np.unique(test_set["section"], return_counts=True)

# np unique returns sorted unique elements
assert np.array_equal(z_train[0], z_test[0])

# plot grouped bar chart
fig, ax = plt.subplots()
x = np.arange(len(z_train[0]))
width = 0.35
rects1 = ax.bar(x - width / 2, z_train[1], width, label="Train")
rects2 = ax.bar(x + width / 2, z_test[1]*(train_set.__len__()/test_set.__len__()), width, 
                label="Scaled Up Test")
ax.set_ylabel("Count")
ax.set_title("Count by Section")
ax.set_xticks(x)
ax.set_xticklabels(z_train[0])
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xticks(rotation=90)
plt.show()

Observations: We have 24 unique sections with Well, Education, Global Business, Your Money, Economy and Automobiles having lowest occurence. Train dataset and test dataset follow similar pattern for section distribution

## Plot headline length by section

In [ ]:
%%script false --no-raise-error

# Group headline_length_per_section by section
from itertools import groupby

def groupby_section(dataset):
    ln_per_section = list(zip(dataset["section"], [len(headline) for headline in dataset["headline"]]))
    ln_per_section = sorted(ln_per_section, key=lambda x: x[0])
    ln_per_section = [(key, list(group)) for key, group in groupby(ln_per_section, lambda x: x[0])]
    ln_per_section = [(key, [x[1] for x in group]) for key, group in ln_per_section]
    ln_stats_per_section = [(key, np.mean(group), np.std(group)) for key, group in ln_per_section]
    return ln_stats_per_section

train_ln_stats_per_section = groupby_section(train_set)
test_ln_stats_per_section = groupby_section(test_set)

In [ ]:
%%script false --no-raise-error

# Plot train_ln_stats_per_section and test_ln_stats_per_section as grouped bar chart with error bars
fig, ax = plt.subplots()
x = np.arange(len(train_ln_stats_per_section))
width = 0.35
rects1 = ax.bar(x - width / 2, [x[1] for x in train_ln_stats_per_section], width, 
                yerr=[x[2] for x in train_ln_stats_per_section], label="Train")
rects2 = ax.bar(x + width / 2, [x[1] for x in test_ln_stats_per_section], width, 
                yerr=[x[2] for x in test_ln_stats_per_section], label="Test")
ax.set_ylabel("Headline Length")
ax.set_title("Headline Length by Section")
ax.set_xticks(x)
ax.set_xticklabels([x[0] for x in train_ln_stats_per_section])
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xticks(rotation=90)
plt.show()


Observations: No outliers in headline length. We can see that the length of the headline is similar across all sections and lies between ~30 to ~80 characters.

## Sample images

Uncomment to visualize

In [ ]:
%%script false --no-raise-error

imgs = []
for i, section in enumerate(tqdm(sections)):
    indices = random.choices(np.where(np.asarray(train_set["section"]) == section)[0], k=5)
    imgs.append(train_set[indices]["image"])

In [ ]:
%%script false --no-raise-error

# Plot images for each section
fig, axs = plt.subplots(6, 4, figsize=(20, 20))
for i, ax in enumerate(axs.flat):
    ax.imshow(imgs[i][random.randint(0, 4)])
    ax.set_title(sections[i])
    ax.axis("off")
plt.subplots_adjust(wspace=0)
plt.show()

In [ ]:
%%script false --no-raise-error

[img.size for img in train_set[random.sample(range(train_set.__len__()), 10)]["image"]]

Observations: Visualized Images have nytimes labeel showing what year it was printed, and sizes vary across images

# Deberta-Base-Uncased

Based on the sample notebook provided --> https://huggingface.co/datasets/generative-newsai/news-unmasked/blob/main/.extras/helper_notebook.ipynb

In [ ]:
%%script false --no-raise-error

model_name = "mlcorelib/deberta-base-uncased"
unmasker = pipeline('fill-mask', model=model_name)

## Check first 5 rows of the Test dataset

In [ ]:
%%script false --no-raise-error

# Check first 5 rows of the test dataset
sample_data = test_set[:5]
for k, v in sample_data.items():
    print("-"*50)
    print(k)
    print(*v, sep="\n")

## Unmask the sentences

In [ ]:
%%script false --no-raise-error

all_masked_words = []
test_subset = Subset(test_set, range(100))
for each_dict in tqdm(test_subset):
    sentence = each_dict['headline']  # Get the sentence from the dictionary
    image_id = each_dict['image_id']  # Get the image_id from the dictionary
    if "[MASK]" in sentence: # See if it has a [MASK] in headline
        result = unmasker(sentence)  # Unmask the sentence

        # Make a list of indices where [MASK] is present in the sentence
        # If there are more than one [MASK] in the sentence, then add them as separate entries in the result list
        indices = [i for i, x in enumerate(sentence.split()) if x == "[MASK]"]
        if len(indices) > 1:
            masked_word_idx_list = []
            for i, each_result in enumerate(result):
                # Get the top scoring word
                top_word = each_result[0]['token_str']
                all_masked_words.append([image_id, indices[i], top_word])
        else:
            all_masked_words.append([image_id, indices[0], result[0]['token_str']])

final_masked_words = [l[0] for l in all_masked_words]

## Print first 5 rows of the masked words list

In [ ]:
%%script false --no-raise-error

print(final_masked_words[:5])

## Save the results as a dataframe and print first 5 rows of the dataframe

In [ ]:
%%script false --no-raise-error

# Save the results in a dataframe with column name id,token_index,token
df = pd.DataFrame(all_masked_words, columns=['id', 'token_index', 'token'])
df.head()

## Save the dataframe as a csv file

In [ ]:
%%script false --no-raise-error

df.to_csv('sample_result.csv', index=False)

# Vision Language Models

## Idea: Caption the image
Ask LLM caption + mask sentence. Predict masks

In [ ]:
# divide train_set in train and validation
train_train_set, train_val_set = torch.utils.data.random_split(train_set, 
                                                               [int(0.8 * len(train_set)),
                                                                len(train_set) - int(0.8 * len(train_set))])

In [ ]:
len(train_train_set), len(train_val_set), len(train_set)

### Image Captioning using hf pipelines

#### VIT-GPT2-COCO

In [ ]:
device_id = 0 if torch.cuda.is_available() else -1

In [ ]:
%%script false --no-raise-error

from transformers import pipeline
from torchmetrics.text.rouge import ROUGEScore

captioner = pipeline("image-to-text",model="ydshieh/vit-gpt2-coco-en", device=device_id)
captioner("https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png")
## [{'generated_text': 'two birds are standing next to each other '}]

In [ ]:
%%script false --no-raise-error

# Try captioner on one of the images in the validation set
rouge = ROUGEScore(use_stemmer=True)
sample = train_val_set[random.randint(0, len(train_val_set))]

caption = captioner(sample["image"], max_new_tokens=25)
rouge_score = rouge(caption[0]['generated_text'], sample["headline"])
plt.imshow(sample["image"])
print("generated_text: " + caption[0]['generated_text'], 
      "section: " + sample["section"],  
      "headline: " + sample["headline"], 
      "rouge1_fmeasure: " + str(rouge_score['rouge1_fmeasure']),
      "rouge2_fmeasure: " + str(rouge_score['rouge2_fmeasure']),
      "rougeL_fmeasure: " + str(rouge_score['rougeL_fmeasure']),
      "rougeLsum_fmeasure: " + str(rouge_score['rougeLsum_fmeasure']),
      sep="\n")

In [ ]:
%%script false --no-raise-error

# rouge1_fmeasure across validation subset
rouge1_fmeasure = []
train_val_subset = Subset(train_val_set, range(100))
for data in tqdm(train_val_subset):
    caption = captioner(data["image"], max_new_tokens=25)
    rouge_score = rouge(caption[0]['generated_text'], data["headline"])
    rouge1_fmeasure.append(rouge_score['rouge1_fmeasure'])

In [ ]:
%%script false --no-raise-error

np.mean(rouge1_fmeasure), np.std(rouge1_fmeasure)

#### BLIP image captioning large

In [ ]:
captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-large", device=device_id)
captioner("https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png")
## VIT-GPT2 output: ## [{'generated_text': 'two birds are standing next to each other '}]
## BLIP output: [{'generated_text': 'there are two parrots that are standing next to each other'}]
## Observations: BLIP output seems more detailed

In [ ]:
%%script false --no-raise-error

# Try captioner on one of the images in the validation set
rouge = ROUGEScore(use_stemmer=True)
sample = train_val_set[random.randint(0, len(train_val_set))]

caption = captioner(sample["image"], max_new_tokens=25)
rouge_score = rouge(caption[0]['generated_text'], sample["headline"])
plt.imshow(sample["image"])
print("generated_text: " + caption[0]['generated_text'], 
      "section: " + sample["section"],  
      "headline: " + sample["headline"], 
      "rouge1_fmeasure: " + str(rouge_score['rouge1_fmeasure']),
      "rouge2_fmeasure: " + str(rouge_score['rouge2_fmeasure']),
      "rougeL_fmeasure: " + str(rouge_score['rougeL_fmeasure']),
      "rougeLsum_fmeasure: " + str(rouge_score['rougeLsum_fmeasure']),
      sep="\n")

In [ ]:
%%script false --no-raise-error

# rouge1_fmeasure across validation subset
rouge1_fmeasure = []
train_val_subset = Subset(train_val_set, range(100))
for data in tqdm(train_val_subset):
    caption = captioner(data["image"], max_new_tokens=25)
    rouge_score = rouge(caption[0]['generated_text'], data["headline"])
    rouge1_fmeasure.append(rouge_score['rouge1_fmeasure'])

In [ ]:
%%script false --no-raise-error

np.mean(rouge1_fmeasure), np.std(rouge1_fmeasure)

# VIT-GPT2 scores: (0.06309018, 0.07374029)
# BLIP scores: (0.06341785, 0.069144554)

Observations:

Inference takes longer than VIT-GPT2 (4:47 as compared to 1:09 for inital 100 samples). Eye balling the results, BLIP image captioning large seems to be more specific to the image. VIT-GPT2 seems to be more generic. rouge score is slightly better for BLIP image captioning large for the first 100 samples. Std dev is also lower for BLIP image captioning large.

Conclusions:

Can try both models and see which gives better results for masked word prediction.

### Fill-Mask Pipeline

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="distilroberta-base", device=device_id)
unmasker("Paris is the <mask> of France.")

# [{'score': 0.7, 'sequence': 'Paris is the capital of France.'},
# {'score': 0.2, 'sequence': 'Paris is the birthplace of France.'},
# {'score': 0.1, 'sequence': 'Paris is the heart of France.'}]

In [ ]:
%%script false --no-raise-error

sample = train_val_set[random.randint(0, len(train_val_set))]

# replace random word with <mask>
masked_sample = sample["headline"].split()
rand_index = random.randint(0, len(masked_sample)-1)
print(f"rand index: {rand_index}, len of headline: {len(masked_sample)}")

masked_sample[rand_index] = "<mask>"
masked_sample = " ".join(masked_sample)
sample["headline"], masked_sample

In [ ]:
%%script false --no-raise-error

unmasked = unmasker(masked_sample)

# compute rouge score with original headline
rouge = ROUGEScore(use_stemmer=True)
print("original headline: " + sample["headline"], "section: " + sample["section"], sep="\n")
for i in range(len(unmasked)):
    rouge_score = rouge(unmasked[i]['sequence'], sample["headline"])
    print("generated_text: " + unmasked[i]['sequence'],  
          "rouge1_fmeasure: " + str(rouge_score['rouge1_fmeasure']),
          "rouge2_fmeasure: " + str(rouge_score['rouge2_fmeasure']),
          "rougeL_fmeasure: " + str(rouge_score['rougeL_fmeasure']),
          "rougeLsum_fmeasure: " + str(rouge_score['rougeLsum_fmeasure']),
          sep="\n")

### Fill mask pipeline with caption as prompt

In [ ]:
def randomMask(input, mask_token="<mask>"):
    masked_input = input.split()
    rand_index = random.randint(0, len(masked_input)-1)
    masked_input[rand_index] = mask_token
    masked_input = " ".join(masked_input)
    return masked_input

In [ ]:
# %%script false --no-raise-error

sample = train_val_set[random.randint(0, len(train_val_set))]
masked_sample = randomMask(sample["headline"])
sample["headline"], masked_sample

PADDING = "[ pad ]"
caption = captioner(sample["image"], max_new_tokens=25)
unmasked = unmasker(masked_sample)
prompted_unmasked = unmasker(caption[0]['generated_text'] + PADDING + masked_sample)

plt.imshow(sample["image"])
print("original headline: " + sample["headline"], "section: " + sample["section"], sep="\n")
print("caption: " + caption[0]['generated_text'], "masked_sample: " + masked_sample, 
      "unmasked: " + unmasked[0]['sequence'], 
      "prompted_unmasked: " + prompted_unmasked[0]['sequence'].split(PADDING)[1],
        sep="\n")

In [ ]:
%%script false --no-raise-error

# What percentage improves with prompting?

train_val_subset = Subset(train_val_set, range(100))
rouge1_fmeasure_prompted = []
rouge1_fmeasure_unprompted = []
for data in tqdm(train_val_subset):
    caption = captioner(data["image"], max_new_tokens=25)
    
    masked_sample = randomMask(data["headline"])
    
    unmasked = unmasker(masked_sample)
    prompted_unmasked = unmasker(caption[0]['generated_text'] + PADDING + masked_sample)
    rouge1_fmeasure_prompted.append(rouge(prompted_unmasked[0]['sequence'].split(PADDING)[1], 
                                          data["headline"])['rouge1_fmeasure'])
    rouge1_fmeasure_unprompted.append(rouge(unmasked[0]['sequence'],
                                            data["headline"])['rouge1_fmeasure'])

In [ ]:
%%script false --no-raise-error

rouge1_fmeasure_prompted = np.array(rouge1_fmeasure_prompted)
rouge1_fmeasure_unprompted = np.array(rouge1_fmeasure_unprompted)
print(np.mean(rouge1_fmeasure_prompted), np.std(rouge1_fmeasure_prompted), 
      np.mean(rouge1_fmeasure_unprompted), np.std(rouge1_fmeasure_unprompted))
print(np.mean(rouge1_fmeasure_prompted > rouge1_fmeasure_unprompted))

#### Running on validation set

In [ ]:
train_val_set.__getitem__(0)

In [ ]:
%%script false --no-raise-error

from torch.utils.data import Subset
from torch.utils.data import DataLoader

all_masked_words = []
# train_val_loader = DataLoader(train_val_set, batch_size=1, shuffle=False)
for each_dict in tqdm(train_val_set):
    original = each_dict['headline'].split(' ')  # Get the original sentence from the dictionary
    image = each_dict['image']  # Get the image from the dictionary
    sentence = each_dict['masked_headline']  # Get the masked_headline from the dictionary
    image_id = each_dict['image_id']  # Get the image_id from the dictionary
    if "[MASK]" in sentence: # See if it has a [MASK] in headline
        sentence = sentence.replace("[MASK]", "<mask>")
        caption = captioner(image, max_new_tokens=25)
        result = unmasker(caption[0]['generated_text'] + PADDING + sentence)
        
        # result = prompted_unmasked[0]['sequence'].split(PADDING)[1]

        # Make a list of indices where [MASK] is present in the sentence
        # If there are more than one [MASK] in the sentence, then add them as separate entries in the result list
        indices = [i for i, x in enumerate(sentence.split()) if x == "<mask>"]
        if len(indices) > 1:
            masked_word_idx_list = []
            for i, each_result in enumerate(result):
                # Get the top scoring word
                top_word = each_result[0]['token_str']
                all_masked_words.append([image_id, indices[i], top_word, original[indices[i]]])
        else:
            all_masked_words.append([image_id, indices[0], result[0]['token_str'], original[indices[0]]])

final_masked_words = [l[0] for l in all_masked_words]

In [ ]:
%%script false --no-raise-error

# Compute cosine similarity using en_core_web_lg
import spacy
from scipy.spatial.distance import cosine

nlp = spacy.load("en_core_web_lg")

cosine_sim_threshold = 0.5
num_correct = 0
for each_masked_word in tqdm(all_masked_words):
    # Get the top scoring word
    top_word = each_masked_word[2]
    # Get the original word
    original_word = each_masked_word[3]
    # Compute cosine similarity
    similarity = nlp(top_word).similarity(nlp(original_word))
    if similarity >= cosine_sim_threshold:
        num_correct += 1

accuracy = num_correct / len(all_masked_words) * 100
print(accuracy)

#### Running on test set

In [ ]:
%%script false --no-raise-error

mask_token = "[MASK]"
test_indices_with_mask = [i for i, x in enumerate(test_set["headline"]) if mask_token in x]

In [ ]:
%%script false --no-raise-error

from torch.utils.data import Subset

all_masked_words = []
test_subset = Subset(test_set, test_indices_with_mask)
for each_dict in tqdm(test_subset):
    image = each_dict['image']  # Get the image from the dictionary
    sentence = each_dict['headline']  # Get the sentence from the dictionary
    image_id = each_dict['image_id']  # Get the image_id from the dictionary
    if "[MASK]" in sentence: # See if it has a [MASK] in headline
        sentence = sentence.replace("[MASK]", "<mask>")
        caption = captioner(image, max_new_tokens=25)
        result = unmasker(caption[0]['generated_text'] + PADDING + sentence)
        
        # result = prompted_unmasked[0]['sequence'].split(PADDING)[1]

        # Make a list of indices where [MASK] is present in the sentence
        # If there are more than one [MASK] in the sentence, then add them as separate entries in the result list
        indices = [i for i, x in enumerate(sentence.split()) if x == "<mask>"]
        if len(indices) > 1:
            masked_word_idx_list = []
            for i, each_result in enumerate(result):
                # Get the top scoring word
                top_word = each_result[0]['token_str']
                all_masked_words.append([image_id, indices[i], top_word])
        else:
            all_masked_words.append([image_id, indices[0], result[0]['token_str']])

final_masked_words = [l[0] for l in all_masked_words]

In [ ]:
%%script false --no-raise-error

len(final_masked_words), np.unique(final_masked_words).shape

In [ ]:
%%script false --no-raise-error

df = pd.DataFrame(all_masked_words, columns=['id', 'token_index', 'token'])
df.head()

In [ ]:
%%script false --no-raise-error

df.to_csv('sample_result.csv', index=False)

### Fill Mask with caption + section as prompt

In [ ]:
%%script false --no-raise-error

sample = train_val_set[random.randint(0, len(train_val_set))]
masked_sample = randomMask(sample["headline"])
sample["headline"], masked_sample

PADDING = "[ pad ]"
caption = captioner(sample["image"], max_new_tokens=25)
unmasked = unmasker(masked_sample)
prompted_unmasked = unmasker(caption[0]['generated_text'] + PADDING + masked_sample)
section_prompted_unmasked = unmasker(sample["section"] + PADDING + 
                                     caption[0]['generated_text'] + PADDING + masked_sample)

plt.imshow(sample["image"])
print("original headline: " + sample["headline"], "section: " + sample["section"], sep="\n")
print("caption: " + caption[0]['generated_text'], "masked_sample: " + masked_sample, 
      "unmasked: " + unmasked[0]['sequence'], 
      "prompted_unmasked: " + prompted_unmasked[0]['sequence'].split(PADDING)[1],
      "section_prompted_unmasked: " + section_prompted_unmasked[0]['sequence'].split(PADDING)[2],
        sep="\n")

In [ ]:
%%script false --no-raise-error

# What percentage improves with adding section as a prompt as well?

train_val_subset = Subset(train_val_set, range(100))
rouge1_fmeasure_prompted = []
rouge1_fmeasure_unprompted = []
rouge1_fmeasure_section_prompted = []
for data in tqdm(train_val_subset):
    caption = captioner(data["image"], max_new_tokens=25)
    
    masked_sample = randomMask(data["headline"])
    
    unmasked = unmasker(masked_sample)
    prompted_unmasked = unmasker(caption[0]['generated_text'] + PADDING + masked_sample)
    section_prompted_unmasked = unmasker(data["section"] + PADDING +
                                            caption[0]['generated_text'] + PADDING + masked_sample)
    rouge1_fmeasure_prompted.append(rouge(prompted_unmasked[0]['sequence'].split(PADDING)[1], 
                                          data["headline"])['rouge1_fmeasure'])
    rouge1_fmeasure_unprompted.append(rouge(unmasked[0]['sequence'],
                                            data["headline"])['rouge1_fmeasure'])
    rouge1_fmeasure_section_prompted.append(rouge(section_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                                            data["headline"])['rouge1_fmeasure'])

In [ ]:
%%script false --no-raise-error
rouge1_fmeasure_prompted = np.array(rouge1_fmeasure_prompted)
rouge1_fmeasure_unprompted = np.array(rouge1_fmeasure_unprompted)
rouge1_fmeasure_section_prompted = np.array(rouge1_fmeasure_section_prompted)

print("prompted mean: ", np.mean(rouge1_fmeasure_prompted), "\n",
      "prompted std: ", np.std(rouge1_fmeasure_prompted), "\n",
      "unprompted mean: ", np.mean(rouge1_fmeasure_unprompted), "\n",
      "unprompted std: ", np.std(rouge1_fmeasure_unprompted), "\n",
      "section_prompted mean: ", np.mean(rouge1_fmeasure_section_prompted), "\n",
      "section_prompted std: ", np.std(rouge1_fmeasure_section_prompted))
print("prompted > unprompted: ", np.mean(rouge1_fmeasure_prompted > rouge1_fmeasure_unprompted) - 
      np.mean(rouge1_fmeasure_prompted < rouge1_fmeasure_unprompted), "\n",
      "section_prompted > unprompted: ", np.mean(rouge1_fmeasure_section_prompted > rouge1_fmeasure_unprompted) - 
      np.mean(rouge1_fmeasure_section_prompted < rouge1_fmeasure_unprompted), "\n",
      "prompted > section_prompted: ", np.mean(rouge1_fmeasure_prompted > rouge1_fmeasure_section_prompted) - 
      np.mean(rouge1_fmeasure_prompted < rouge1_fmeasure_section_prompted))

Observations:
- Adding section additionaly as the prompt sometimes improves the performance of the model.

Nothing conclusive can be said about the performance of the model with section as prompt. It seems to be working better for some sections and worse for others.

Maybe can add a description to section

In [ ]:
%%script false --no-raise-error
score1 = -1
score2 = -1
score3 = -1
score4 = -1
count = 1000

while (score1 <= 0 or score3 <= 0) and (score2 <= 0 or score4 <= 0) and count > 0:
      count -= 1
      sample = train_val_set[random.randint(0, len(train_val_set))]
      masked_sample = randomMask(sample["headline"])
      sample["headline"], masked_sample

      PADDING = "[ pad ]"
      caption = captioner(sample["image"], max_new_tokens=25)
      unmasked = unmasker(masked_sample)
      prompted_unmasked = unmasker(caption[0]['generated_text'] + PADDING + masked_sample)
      section_prompted_unmasked = unmasker(sample["section"] + PADDING + 
                                          caption[0]['generated_text'] + PADDING + masked_sample)
      desc_prompted_unmasked = unmasker("The context for the caption is {0}.".format(sample["section"]) + PADDING +
                                    caption[0]['generated_text'] + PADDING + masked_sample)
      score1 = rouge(desc_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                     sample["headline"])['rouge1_fmeasure'] - rouge(unmasked[0]['sequence'],
                                                                    sample["headline"])['rouge1_fmeasure']
      score2 = rouge(section_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                     sample["headline"])['rouge1_fmeasure'] - rouge(unmasked[0]['sequence'],
                                                                    sample["headline"])['rouge1_fmeasure']
      score3 = rouge(desc_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                     sample["headline"])['rouge1_fmeasure'] - rouge(
                           prompted_unmasked[0]['sequence'].split(PADDING)[1],
                           sample["headline"])['rouge1_fmeasure']
      score4 = rouge(section_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                        sample["headline"])['rouge1_fmeasure'] - rouge(
                              prompted_unmasked[0]['sequence'].split(PADDING)[1],
                              sample["headline"])['rouge1_fmeasure']

print(score1, score2, score3, score4)
plt.imshow(sample["image"])
print("original headline: " + sample["headline"], "section: " + sample["section"], sep="\n")
print("caption: " + caption[0]['generated_text'], "masked_sample: " + masked_sample, 
      "unmasked: " + unmasked[0]['sequence'], 
      "prompted_unmasked: " + prompted_unmasked[0]['sequence'].split(PADDING)[1],
      "section_prompted_unmasked: " + section_prompted_unmasked[0]['sequence'].split(PADDING)[2],
      "desc_prompted_unmasked: " + desc_prompted_unmasked[0]['sequence'].split(PADDING)[2],
      sep="\n")

In [ ]:
%%script false --no-raise-error

# What percentage improves with adding section as a prompt as well?

train_val_subset = Subset(train_val_set, range(100))
rouge1_fmeasure_prompted = []
rouge1_fmeasure_unprompted = []
rouge1_fmeasure_section_prompted = []
rouge1_fmeasure_desc_prompted = []
for data in tqdm(train_val_subset):
    caption = captioner(data["image"], max_new_tokens=25)
    
    masked_sample = randomMask(data["headline"])
    
    unmasked = unmasker(masked_sample)
    prompted_unmasked = unmasker(caption[0]['generated_text'] + PADDING + masked_sample)
    section_prompted_unmasked = unmasker(data["section"] + PADDING +
                                            caption[0]['generated_text'] + PADDING + masked_sample)
    desc_prompted_unmasked = unmasker("The context for the caption is {0}.".format(data["section"]) + PADDING +
                                            caption[0]['generated_text'] + PADDING + masked_sample)
    rouge1_fmeasure_prompted.append(rouge(prompted_unmasked[0]['sequence'].split(PADDING)[1], 
                                          data["headline"])['rouge1_fmeasure'])
    rouge1_fmeasure_unprompted.append(rouge(unmasked[0]['sequence'],
                                            data["headline"])['rouge1_fmeasure'])
    rouge1_fmeasure_section_prompted.append(rouge(section_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                                            data["headline"])['rouge1_fmeasure'])
    rouge1_fmeasure_desc_prompted.append(rouge(desc_prompted_unmasked[0]['sequence'].split(PADDING)[2],
                                            data["headline"])['rouge1_fmeasure'])

In [ ]:
%%script false --no-raise-error

rouge1_fmeasure_prompted = np.array(rouge1_fmeasure_prompted)
rouge1_fmeasure_unprompted = np.array(rouge1_fmeasure_unprompted)
rouge1_fmeasure_section_prompted = np.array(rouge1_fmeasure_section_prompted)
rouge1_fmeasure_desc_prompted = np.array(rouge1_fmeasure_desc_prompted)

print("prompted mean: ", np.mean(rouge1_fmeasure_prompted), "\n",
      "prompted std: ", np.std(rouge1_fmeasure_prompted), "\n",
      "unprompted mean: ", np.mean(rouge1_fmeasure_unprompted), "\n",
      "unprompted std: ", np.std(rouge1_fmeasure_unprompted), "\n",
      "section_prompted mean: ", np.mean(rouge1_fmeasure_section_prompted), "\n",
      "section_prompted std: ", np.std(rouge1_fmeasure_section_prompted), "\n",
      "desc_prompted mean: ", np.mean(rouge1_fmeasure_desc_prompted), "\n",
      "desc_prompted std: ", np.std(rouge1_fmeasure_desc_prompted))
print("prompted > unprompted: ", np.mean(rouge1_fmeasure_prompted > rouge1_fmeasure_unprompted) - 
      np.mean(rouge1_fmeasure_prompted < rouge1_fmeasure_unprompted), "\n",
      "section_prompted > unprompted: ", np.mean(rouge1_fmeasure_section_prompted > rouge1_fmeasure_unprompted) - 
      np.mean(rouge1_fmeasure_section_prompted < rouge1_fmeasure_unprompted), "\n",
      "prompted > section_prompted: ", np.mean(rouge1_fmeasure_prompted > rouge1_fmeasure_section_prompted) - 
      np.mean(rouge1_fmeasure_prompted < rouge1_fmeasure_section_prompted), "\n",
      "desc_prompted > unprompted: ", np.mean(rouge1_fmeasure_desc_prompted > rouge1_fmeasure_unprompted) -
      np.mean(rouge1_fmeasure_desc_prompted < rouge1_fmeasure_unprompted), "\n",
      "desc_prompted > prompted: ", np.mean(rouge1_fmeasure_desc_prompted > rouge1_fmeasure_prompted) -
      np.mean(rouge1_fmeasure_desc_prompted < rouge1_fmeasure_prompted), "\n",
      "desc_prompted > section_prompted: ", np.mean(rouge1_fmeasure_desc_prompted > rouge1_fmeasure_section_prompted) -
      np.mean(rouge1_fmeasure_desc_prompted < rouge1_fmeasure_section_prompted))

Observations:
- Adding descriptive section as a prompt sometime shows improvements over unprompted and prompted.

Need to check what works better on what kind of random masking

Can we just take all of them and ensemble them somehow?

## BLIP

### BLIP conditional generation

In [ ]:
## Sample BLIP

import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
plt.imshow(raw_image)

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt").to(device)

out = model.generate(**inputs, max_new_tokens=64)
print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to(device)

out = model.generate(**inputs, max_new_tokens=64)
print(processor.decode(out[0], skip_special_tokens=True))

# Masked image caption
inputs = processor(raw_image, "woman sitting on the beach with her [MASK]", return_tensors="pt").to(device)

out = model.generate(**inputs, max_new_tokens=64)
print(processor.decode(out[0], skip_special_tokens=True))
generated_text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
print(generated_text)

Observations: Seems to complete the sentence but not able to fill the masks

#### Finetuning BLIP

Finetunig BLIP with MLM Head like Deberta

##### BLIPFor MLM from scratch 

In [ ]:
# from transformers.models.blip.modeling_blip import BlipPreTrainedModel, BlipModel
# from transformers.models.deberta.modeling_deberta import DebertaOnlyMLMHead
# from typing import Optional, Union, Tuple
# from torch.nn import CrossEntropyLoss, Module
# from transformers.modeling_outputs import MaskedLMOutput

# class BLIPForMLM(BlipPreTrainedModel):
#     _keys_to_ignore_on_load_unexpected = [r"pooler"]
#     _keys_to_ignore_on_load_missing = [r"position_ids", r"predictions.decoder.bias", "cls.predictions.decoder.weight"]

#     def __init__(self, config):
#         super().__init__(config)

#         self.deberta = BlipModel(config)
#         self.cls = BlipTextOnlyMLMHead(config)

#         # Initialize weights and apply final processing
#         self.post_init()

#     def get_output_embeddings(self):
#         return self.cls.predictions.decoder

#     def set_output_embeddings(self, new_embeddings):
#         self.cls.predictions.decoder = new_embeddings
        
#     def forward(
#         self,
#         input_ids: Optional[torch.Tensor] = None,
#         attention_mask: Optional[torch.Tensor] = None,
#         token_type_ids: Optional[torch.Tensor] = None,
#         position_ids: Optional[torch.Tensor] = None,
#         inputs_embeds: Optional[torch.Tensor] = None,
#         labels: Optional[torch.Tensor] = None,
#         output_attentions: Optional[bool] = None,
#         output_hidden_states: Optional[bool] = None,
#         return_dict: Optional[bool] = None,
#     ) -> Union[Tuple, MaskedLMOutput]:
#         r"""
#         labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
#             Labels for computing the masked language modeling loss. Indices should be in `[-100, 0, ...,
#             config.vocab_size]` (see `input_ids` docstring) Tokens with indices set to `-100` are ignored (masked), the
#             loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`
#         """

#         return_dict = return_dict if return_dict is not None else self.config.use_return_dict

#         outputs = self.deberta(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             position_ids=position_ids,
#             inputs_embeds=inputs_embeds,
#             output_attentions=output_attentions,
#             output_hidden_states=output_hidden_states,
#             return_dict=return_dict,
#         )

#         sequence_output = outputs[0]
#         prediction_scores = self.cls(sequence_output)

#         masked_lm_loss = None
#         if labels is not None:
#             loss_fct = CrossEntropyLoss()  # -100 index = padding token
#             masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

#         if not return_dict:
#             output = (prediction_scores,) + outputs[1:]
#             return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output

#         return MaskedLMOutput(
#             loss=masked_lm_loss,
#             logits=prediction_scores,
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#         )


#### Training

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

In [ ]:
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
image = Image.open(requests.get(url, stream=True).raw)
text = "a picture of a [MASK] and her dog on the beach"

In [ ]:
%%script false --no-raise-error

# inputs = processor(images=image, text=text, return_tensors="pt").to(device)
# print("inputs: ", processor.decode(inputs['input_ids'][0]))

# Training
# for p in model.parameters():
#     p.requires_grad = True
from tqdm import trange
for i in trange(100):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
    inputs = processor(images=image, text=text, return_tensors="pt")
    labels = processor(text="a picture of a woman and her dog on the beach", return_tensors="pt").input_ids

    inputs["labels"] = labels
    outputs = model(**inputs)
    loss = outputs.loss
    print(loss.item())
    loss.backward()
    optimizer.step()

# outputs = model.generate(**inputs, max_new_tokens=64)
# print(processor.decode(outputs[0], skip_special_tokens=True))
# outputs = model(**inputs)
# # print(processor.decode(outputs.decoder_logits[0,0,:]))
# plt.imshow(image)

In [ ]:
%%script false --no-raise-error

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=64)
print(processor.decode(outputs[0]))

In [ ]:
%%script false --no-raise-error

outputs.decoder_logits.shape

In [ ]:
%%script false --no-raise-error

# retrieve index of [MASK]
inputs = processor(images=image, text=text, return_tensors="pt").to(device)
mask_token_index = (inputs.input_ids == processor.tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

outputs = model(**inputs)
logits = outputs.decoder_logits
print(inputs.input_ids.shape, logits.shape, mask_token_index)
print(processor.decode(inputs.input_ids[0,mask_token_index]))
predicted_token_id = logits[0, mask_token_index-1].argmax(axis=-1)
print(processor.decode(predicted_token_id))
print(processor.decode(inputs.input_ids[0]))
print(processor.decode(logits[0].argmax(axis=-1)))

Observation: BLIPForMLM is based on BERT but they shift the output to predict next word. Hence the masked word output in shifted as well

In [ ]:
%%script false --no-raise-error

# retrieve index of [MASK]
text = "a [MASK] of a [MASK] and her [MASK] on the [MASK]"
inputs = processor(images=image, text=text, return_tensors="pt").to(device)
mask_token_index = (inputs.input_ids == processor.tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

outputs = model(**inputs)
logits = outputs.decoder_logits
print(inputs.input_ids.shape, logits.shape, mask_token_index)
print(processor.decode(inputs.input_ids[0,mask_token_index]))
predicted_token_id = logits[0, mask_token_index-1].argmax(axis=-1)
print(processor.decode(predicted_token_id))
print(processor.decode(inputs.input_ids[0]))
print(processor.decode(logits[0].argmax(axis=-1)))

#### Save a Processed Dataset

In [ ]:
len(train_train_set), len(train_val_set), len(test_set)

In [ ]:
%%script false --no-raise-error
randomMask(train_train_set[0]['headline'])

In [ ]:
def batchedRandomMask(examples):
    headlines = [randomMask(headline) for headline in examples['headline']]
    return {'masked_headline': headlines}

In [ ]:
%%script false --no-raise-error
chunked_dataset = train_set.map(batchedRandomMask, batched=True)

In [ ]:
%%script false --no-raise-error
rindex = random.randint(0, len(chunked_dataset))
chunked_dataset[rindex]['masked_headline'], chunked_dataset[rindex]['headline']

In [ ]:
%%script false --no-raise-error
chunked_dataset.save_to_disk("./processed_data")

#### BLIP Baseline

In [ ]:
%%script false --no-raise-error
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

In [ ]:
%%script false --no-raise-error
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
image = Image.open(requests.get(url, stream=True).raw)
text = "a picture of a [MASK] and her [MASK] on the beach"

In [ ]:
%%script false --no-raise-error
def unmask(text, image, device):
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    mask_token_index = (inputs.input_ids == processor.tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

    outputs = model(**inputs)
    logits = outputs.decoder_logits
    predicted_token_id = logits[0, mask_token_index-1].argmax(axis=-1)
    return processor.decode(predicted_token_id)

result = unmask(text, image, device)
print(result.split(' '))

In [ ]:
%%script false --no-raise-error

from torch.utils.data import DataLoader

all_masked_words = []
# val_loader = DataLoader(train_val_set, batch_size=10, shuffle=False)
# train_val_subset = torch.utils.data.Subset(train_val_set, range(100))
for each_dict in tqdm(train_val_set):
    original = each_dict['headline'].split(' ')  # Get the original sentence from the dictionary
    sentence = each_dict['masked_headline']  # Get the sentence from the dictionary
    image_id = each_dict['image_id']  # Get the image_id from the dictionary
    image = each_dict['image']  # Get the image from the dictionary
    if "[MASK]" in sentence: # See if it has a [MASK] in headline
        result = unmask(sentence, image, device).split(' ')

        # Make a list of indices where [MASK] is present in the sentence
        # If there are more than one [MASK] in the sentence, then add them as separate entries in the result list
        indices = [i for i, x in enumerate(sentence.split()) if x == "[MASK]"]
        if len(indices) > 1:
            masked_word_idx_list = []
            for i, each_result in enumerate(result):
                # Get the top scoring word
                all_masked_words.append([image_id, indices[i], each_result, original[indices[i]]])
        else:
            all_masked_words.append([image_id, indices[0], result[0], original[indices[0]]])

final_masked_words = [l[0] for l in all_masked_words]

In [ ]:
%%script false --no-raise-error
all_masked_words[:10]

In [ ]:
%%script false --no-raise-error

# Compute cosine similarity using en_core_web_lg
import spacy
from scipy.spatial.distance import cosine

nlp = spacy.load("en_core_web_lg")

cosine_sim_threshold = 0.5
num_correct = 0
for each_masked_word in tqdm(all_masked_words):
    # Get the top scoring word
    top_word = each_masked_word[2]
    # Get the original word
    original_word = each_masked_word[3]
    # Compute cosine similarity
    similarity = nlp(top_word).similarity(nlp(original_word))
    if similarity >= cosine_sim_threshold:
        num_correct += 1

accuracy = num_correct / len(all_masked_words) * 100
print(accuracy)

#### Finetune BLIP

In [ ]:
%%script false --no-raise-error
from torch.utils.data import DataLoader
from torchvision.transforms.functional import pil_to_tensor
from torch.utils.data.dataloader import default_collate

def custom_collate(original_batch):
    filtered_data = {}

    for item in original_batch:
        image, text = item['image'], item['masked_headline']
        original = item['headline']
        inputs = processor(images=image, text=text, return_tensors="pt")
        inputs["labels"] = processor(text=original, return_tensors="pt").input_ids
        for k, v in inputs.items():
            if k not in filtered_data:
                filtered_data[k] = []
            filtered_data[k].append(v)
    
    # for k, v in filtered_data.items():
    #     filtered_data[k] = default_collate(v)

    return filtered_data
    # return default_collate(filtered_data), default_collate(filtered_target)

In [ ]:
%%script false --no-raise-error
temp = Subset(train_train_set, range(100))
temp_loader = DataLoader(temp, batch_size=10, shuffle=False, collate_fn=custom_collate)

In [ ]:
%%script false --no-raise-error
for i, batch in enumerate(temp_loader):
    print(batch.keys(), len(batch['input_ids']), len(batch['labels']))
    break
    # print(batch['image'].shape, batch['image_id'], batch['headline'])

In [ ]:
%%script false --no-raise-error
from tqdm import trange

temp = Subset(train_train_set, range(100))
# temp_loader = DataLoader(temp, batch_size=10, shuffle=False, collate_fn=custom_collate)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
losses = []
epochs = 10
for epoch in trange(epochs):
    for i, data in enumerate(tqdm(temp, leave=False)):
        model.train()
        image, text = data['image'], data['masked_headline']
        original = data['headline']
        inputs = processor(images=image, text=text, return_tensors="pt")
        labels = processor(text=original, return_tensors="pt").input_ids

        inputs["labels"] = labels
        if inputs['input_ids'].shape != labels.shape:
            # print("Skipping")
            continue
        inputs = inputs.to(device)
        outputs = model(**inputs)
        loss = outputs.loss
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"Loss: {np.mean(losses)}")
            losses = []

# outputs = model.generate(**inputs, max_new_tokens=64)
# print(processor.decode(outputs[0], skip_special_tokens=True))
# outputs = model(**inputs)
# # print(processor.decode(outputs.decoder_logits[0,0,:]))
# plt.imshow(image)

In [ ]:
%%script false --no-raise-error
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
image = Image.open(requests.get(url, stream=True).raw)
text = "a picture of a [MASK] and her [MASK] on the beach"

result = unmask(text, image, device)
print(text, result, sep='\n')

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
mask_token_index = (inputs.input_ids == processor.tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

outputs = model(**inputs)
logits = outputs.decoder_logits
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
print(processor.decode(logits.argmax(axis=-1)[0]))

In [ ]:
%%script false --no-raise-error
from torch.utils.data import DataLoader

all_masked_words = []
# val_loader = DataLoader(train_val_set, batch_size=10, shuffle=False)
# train_val_subset = torch.utils.data.Subset(train_val_set, range(100))
for each_dict in tqdm(train_val_set):
    original = each_dict['headline'].split(' ')  # Get the original sentence from the dictionary
    sentence = each_dict['masked_headline']  # Get the sentence from the dictionary
    image_id = each_dict['image_id']  # Get the image_id from the dictionary
    image = each_dict['image']  # Get the image from the dictionary
    if "[MASK]" in sentence: # See if it has a [MASK] in headline
        result = unmask(sentence, image, device).split(' ')

        # Make a list of indices where [MASK] is present in the sentence
        # If there are more than one [MASK] in the sentence, then add them as separate entries in the result list
        indices = [i for i, x in enumerate(sentence.split()) if x == "[MASK]"]
        if len(indices) > 1:
            masked_word_idx_list = []
            for i, each_result in enumerate(result):
                # Get the top scoring word
                all_masked_words.append([image_id, indices[i], each_result, original[indices[i]]])
        else:
            all_masked_words.append([image_id, indices[0], result[0], original[indices[0]]])

final_masked_words = [l[0] for l in all_masked_words]

In [ ]:
%%script false --no-raise-error

# Compute cosine similarity using en_core_web_lg
import spacy
from scipy.spatial.distance import cosine

nlp = spacy.load("en_core_web_lg")

cosine_sim_threshold = 0.5
num_correct = 0
for each_masked_word in tqdm(all_masked_words):
    # Get the top scoring word
    top_word = each_masked_word[2]
    # Get the original word
    original_word = each_masked_word[3]
    # Compute cosine similarity
    similarity = nlp(top_word).similarity(nlp(original_word))
    if similarity >= cosine_sim_threshold:
        num_correct += 1

accuracy = num_correct / len(all_masked_words) * 100
print(accuracy)

# TODOs
- [Done] Create a masked dataset and save it to run tests
- [Failed] FineTune the BlipForConditionalGeneration on train dataset
    - Run training validation on worst performing examples

Maybe:
- Train a BLIPMLM Head